In [ ]:
from collections import deque

# --- Параметры задачи ---
# Начальное распределение компота по стаканам
INITIAL_STATE = (10, 50, 20, 80, 40)

# Вычисляем целевое состояние
total_volume = sum(INITIAL_STATE)
num_glasses = len(INITIAL_STATE)

if total_volume % num_glasses != 0:
    raise ValueError("Общий объем компота должен делиться на количество стаканов!")

TARGET_VOLUME = total_volume // num_glasses
GOAL_STATE = tuple([TARGET_VOLUME] * num_glasses)

print(f"Начальное состояние: {INITIAL_STATE}")
print(f"Целевое состояние: {GOAL_STATE}")

In [ ]:
def solve_compote_bfs(initial_state, goal_state, target_volume):
    """
    Находит кратчайший путь от начального до целевого состояния с помощью BFS.
    """
    # Очередь для состояний, которые нужно посетить.
    # Храним кортеж (состояние, путь_к_состоянию)
    queue = deque([(initial_state, [initial_state])])
    
    # Множество для уже посещенных состояний, чтобы не ходить по кругу
    visited = {initial_state}
    
    while queue:
        current_state, path = queue.popleft()
        
        # Если мы достигли цели, возвращаем найденный путь
        if current_state == goal_state:
            return path
            
        # --- Генерация всех возможных следующих состояний из текущего ---
        # Это "сердце" алгоритма, реализующее правило "одно поднятие - много розливов"
        for i in range(num_glasses):
            # i - это индекс стакана, который мы "поднимаем" (source glass)
            
            # Если в стакане нечего переливать (он уже на целевом уровне или ниже),
            # то нет смысла его поднимать для розлива.
            if current_state[i] <= target_volume:
                continue

            # Создаем копию состояния для симуляции переливания
            next_state_list = list(current_state)
            
            # Весь излишек из стакана i можно разлить по другим стаканам
            pourable_amount = next_state_list[i] - target_volume
            next_state_list[i] = target_volume # Сразу опустошаем источник до целевого уровня
            
            # Распределяем 'pourable_amount' по стаканам, где не хватает компота
            for j in range(num_glasses):
                if i == j: # Нельзя лить в самого себя
                    continue
                
                # Если компота можно долить и есть что наливать
                if next_state_list[j] < target_volume and pourable_amount > 0:
                    needed = target_volume - next_state_list[j]
                    transfer = min(pourable_amount, needed)
                    
                    next_state_list[j] += transfer
                    pourable_amount -= transfer

            new_state = tuple(next_state_list)
            
            # Если мы получили новое, ранее не посещенное состояние,
            # добавляем его в очередь и в множество посещенных
            if new_state not in visited:
                visited.add(new_state)
                new_path = path + [new_state]
                queue.append((new_state, new_path))
                
    return None # Если очередь опустела, а решение не найдено

In [ ]:
print("--- Поиск оптимального решения ---")
solution_path = solve_compote_bfs(INITIAL_STATE, GOAL_STATE, TARGET_VOLUME)

if solution_path:
    print(f"\nРешение найдено за {len(solution_path) - 1} шаг(а/ов)!\n")
    print("Оптимальная последовательность действий:")
    for i, state in enumerate(solution_path):
        if i == 0:
            print(f"Шаг 0 (Начало): {state}")
        else:
            # Определяем, какой стакан был источником
            prev_state = solution_path[i-1]
            source_glass_index = -1
            for j in range(num_glasses):
                if prev_state[j] > state[j]:
                    source_glass_index = j
                    break
            
            print(f"\nШаг {i}: Поднимаем стакан {source_glass_index + 1} ({prev_state[source_glass_index]} мл)")
            print(f"Результат:     {state}")
            
else:
    print("Не удалось найти решение.")